# Test Assignment. Junior Data Analyst 
## Damir Umetbaev

### ## Выводы по работе в самом низу страницы

Работа разбита на два файла. Второй - Data preparation, его нужно запустить первым. Сделал такую разбивку, чтобы снизить нагрузку на компьютер

In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
df = pd.read_csv('df_parsed.csv')
# delete irrelevant column
df = df.drop(['Unnamed: 0'], axis=1)
# create df for storing results
d = {'City_Rev': [None, None, None], 'City_ECPM': [None, None, None],\
     'OS_Rev': [None, None, None], 'OS_ECPM': [None, None, None]}
df_results = pd.DataFrame(data=d, index=['Int', 'Banner', 'RV'])

## ECPM Analysis

In [ ]:
# add ECPM column (revenue*1000)
df['ECPM'] = df.revenue*1000

### Overall

In [ ]:
fig = px.histogram(df, y='ECPM')
fig.update_yaxes(range=[0, 15])
fig.update_xaxes(range=[0, 300000])
fig.update_layout(title = "ECPM",
                  xaxis_title="Number of cases",
                  yaxis_title="ECPM value")
fig.show()

Из общего распределения eCPM можно видеть что три самых крупных класса локализованы возле 7, возле 2 и возле 0,5 (USD).
Самая показываемая реклама - дешевая реклама.

## Int Ad

Из общего массива данных выделю тот что относится к полноэкранной рекламе

In [ ]:
df_int = df.loc[df['ad_type'] == 'int']

In [ ]:
fig = px.histogram(df_int, y=df_int.ECPM)
fig.update_yaxes(range=[0, 400])
fig.update_xaxes(range=[0, 4000])
fig.update_layout(title = "ECPM int",
                  xaxis_title="Number of cases",
                  yaxis_title="ECPM value")
fig.show()

Из графика видно что есть несколько групп событий, причем ECPM основной массы сейчас в районе 15$, но есть в разы более выгодные группы с ECPM 190$, 170$, 130$

## Анализ по городам 

Суммарные доходы по городам

In [ ]:
df_int_cities_sum_revenue = df_int.groupby(['city'])['revenue'].sum().reset_index(name='revenue')
df_int_cities_sum_revenue = df_int_cities_sum_revenue.sort_values(by='revenue', ascending=False)
Total = df_int_cities_sum_revenue['revenue'].sum()
Subtotal = df_int_cities_sum_revenue['revenue'].iloc[:100,].sum()
df_int_cities_sum_revenue = df_int_cities_sum_revenue.loc[df_int_cities_sum_revenue['revenue'] > 10]

df_results.at['Int','City_Rev'] = (df_int_cities_sum_revenue.loc[df_int_cities_sum_revenue['revenue'].idxmax()])['revenue']

fig = px.bar(df_int_cities_sum_revenue, x='city', y='revenue')
fig.update_layout(title = "Int revenue sum",
                  xaxis_title="City name",
                  yaxis_title="revenue sum")
fig.show()

In [ ]:
Subtotal,Total

В датасете присутствует порядка 900 городов, 18% дохода приходится на первую десятку, 61% на первую сотню, т.е. доход относительно распределен по городам.

Средние значения ECPM по городам

### Про пункт 2 задания. Оценить стат значимость полученных средних значений ECPM.

Я не знаю как оценить стат.значимость для неизвестного мне объекта исследования (полученных средних ECPM), бегло нагуглить как это делается так же не удалось. Я мог бы определить стат значимость зная, к примеру, старое распределение величин (средние значения ECPM по городам и средние отклонения). 

Для приблизительной оценки значимости в данной ситуации я построил график mean ECPM распределенных по размерам выборки.

In [ ]:
# calculation of mean ECPM and count number of events for each city
df_int_cities_mean_ECPM = df_int.groupby('city')['ECPM']\
                        .agg(['mean','size'])\
                        .rename(columns={'mean':'ECPM'})\
                        .reset_index()

fig = px.scatter(df_int_cities_mean_ECPM, x="ECPM", y="size")
fig.update_layout(title = "Mean ECPM to frequency int",
                  xaxis_title="mean ECPM",
                  yaxis_title="Number of cases")
fig.show()

Из графика видно что широкий разброс по mean ECPM (от 0 до 437) существует только при малых (<50) размерах выборки. С увеличением размера выборки вариативность сильно падает. Исходя из общего правила - точность растет с размером выборки, в качестве условной оценки я дам статус неточных всем значениям с размером выборки меньше 80. В дальнейшем я буду использовать это число как планку и отсекать результаты с меньшим размером выборки.

Так же из графика видно, что вариативность стоимости показа рекламы крайне высока, в некоторых случаях в десятки раз. 

Далее я отсортирую результаты по ECPM чтобы выделить наиболее успешные города.

In [ ]:
df_int_cities_mean_ECPM = df_int_cities_mean_ECPM.loc[df_int_cities_mean_ECPM['size'] > 80]
df_int_cities_mean_ECPM = df_int_cities_mean_ECPM.sort_values(by = 'ECPM', ascending=False)

Таким образом из 869 городов осталось лишь 64. 
Для графического отображения выведу первые 8

In [ ]:
df_int_cities_mean_ECPM = df_int_cities_mean_ECPM.loc[df_int_cities_mean_ECPM['ECPM'] > 50]
df_results.at['Int','City_ECPM'] = (df_int_cities_mean_ECPM.loc[df_int_cities_mean_ECPM['ECPM'].idxmax()])['ECPM']

fig = px.bar(df_int_cities_mean_ECPM, x='city', y='ECPM')
fig.update_layout(title = "Int ECPM mean",
                  xaxis_title="City name",
                  yaxis_title="ECPM mean")
fig.show()

Города с наибольшим доходам и города с наиболее дорогими показами рекламы не совпадают. 

## Анализ по версиям ОС

Проверяю общее кол-во версий

In [ ]:
df.drop_duplicates(subset=['os_version']).shape

Версий всего 11

Суммарный доход по версиям

In [ ]:
df_int_os_sum_revenue = df_int.groupby(['os_version'])['revenue'].sum().reset_index(name='revenue')
df_int_os_sum_revenue = df_int_os_sum_revenue.sort_values(by='revenue', ascending=False)

df_results.at['Int','OS_Rev'] = (df_int_os_sum_revenue.loc[df_int_os_sum_revenue['revenue'].idxmax()])['revenue']

fig = px.bar(df_int_os_sum_revenue, x='os_version', y='revenue')
fig.update_layout(title = "Int revenue sum",
                  xaxis_title="os_version name",
                  yaxis_title="revenue sum")
fig.show()

Почти все активные пользователи используют последние 4 версии ОС. Поддержка версий старее девятой это меньше 1% прибыли.

Средние значения ECPM по версиям ОС

In [ ]:
# calculation of mean ECPM and count number of events for each city
df_int_os_mean_ECPM = df_int.groupby('os_version')['ECPM']\
                        .agg(['mean','size'])\
                        .rename(columns={'mean':'ECPM'})\
                        .reset_index()

df_int_os_mean_ECPM = df_int_os_mean_ECPM.loc[df_int_os_mean_ECPM['size'] > 80]
df_int_os_mean_ECPM = df_int_os_mean_ECPM.sort_values(by = 'ECPM', ascending=False)

df_results.at['Int','OS_ECPM'] = (df_int_os_mean_ECPM.loc[df_int_os_mean_ECPM['ECPM'].idxmax()])['ECPM']

fig = px.bar(df_int_os_mean_ECPM, x='os_version', y='ECPM')
fig.update_layout(title = "Int ECPM mean",
                  xaxis_title="os_version name",
                  yaxis_title="ECPM mean")
fig.show()

Чем новее версия ОС тем больше стоит пользователь, прямая корреляция.

## Banner Ad

Из общего массива данных выделю тот что относится к баннерной рекламе

In [ ]:
df_banner = df.loc[df['ad_type'] == 'banner']

In [ ]:
fig = go.Figure(data=[go.Histogram(y=df_banner.ECPM)])
fig.update_yaxes(range=[0, 10])
fig.update_xaxes(range=[0, 140000])
fig.update_layout(title = "ECPM banner",
                  xaxis_title="Number of cases",
                  yaxis_title="ECPM value")
fig.show()

На графике видно три класса событий. Два верхних класса представляют интерес для дальнейшего изучения, т.к. являются в разы более прибыльными.

## Анализ по городам

Суммарные доходы по городам

In [ ]:
df_banner_cities_sum_revenue = df_banner.groupby(['city'])['revenue'].sum().reset_index(name='revenue')
df_banner_cities_sum_revenue = df_banner_cities_sum_revenue.sort_values(by='revenue', ascending=False)
Total = df_banner_cities_sum_revenue['revenue'].sum()
Subtotal = df_banner_cities_sum_revenue['revenue'].iloc[:10,].sum()
df_banner_cities_sum_revenue = df_banner_cities_sum_revenue.loc[df_banner_cities_sum_revenue['revenue'] > 5]

df_results.at['Banner','City_Rev'] = (df_banner_cities_sum_revenue.loc[df_banner_cities_sum_revenue['revenue'].idxmax()])['revenue']

fig = px.bar(df_banner_cities_sum_revenue, x='city', y='revenue')
fig.update_layout(title = "Banner revenue sum",
                  xaxis_title="City name",
                  yaxis_title="revenue sum")
fig.show()

In [ ]:
Subtotal,Total

На первую десятку приходится 21.2% всего дохода, а на первую сотню 68.7%

Средние значения ECPM по городам 

In [ ]:
# calculation of mean ECPM and count number of events for each city
df_banner_cities_mean_ECPM = df_banner.groupby('city')['ECPM']\
                        .agg(['mean','size'])\
                        .rename(columns={'mean':'ECPM'})\
                        .reset_index()

df_banner_cities_mean_ECPM = df_banner_cities_mean_ECPM.sort_values(by = 'size')

df_banner_cities_mean_ECPM = df_banner_cities_mean_ECPM.loc[df_banner_cities_mean_ECPM['size'] > 80]

fig = px.scatter(df_banner_cities_mean_ECPM, x="ECPM", y="size")
fig.update_layout(title = "Mean ECPM to frequency Banner",
                  xaxis_title="mean ECPM",
                  yaxis_title="Number of cases")
fig.show()

Для данного типа рекламы виден сильный разброс в стоимости. Есть несколько городов, цена просмотра рекламы в которых значительно выше чем в остальных.

In [ ]:
df_banner_cities_mean_ECPM = df_banner_cities_mean_ECPM.loc[df_banner_cities_mean_ECPM['ECPM'] > 3]
df_banner_cities_mean_ECPM = df_banner_cities_mean_ECPM.sort_values(by = 'ECPM', ascending=False)
df_results.at['Banner','City_ECPM'] = (df_banner_cities_mean_ECPM.loc[df_banner_cities_mean_ECPM['ECPM'].idxmax()])['ECPM']

fig = px.bar(df_banner_cities_mean_ECPM, x='city', y='ECPM')
fig.update_layout(title = "Banner ECPM mean",
                  xaxis_title="City name",
                  yaxis_title="ECPM mean")
fig.show()

Из графика видно что первая тройка примерно в 2 раза ценнее остальных топовых городов. Преимущество первых трех городов ярко выраженно.

## Анализ по версиям ОС

Суммарный доход по версиям

In [ ]:
df_banner_os_sum_revenue = df_banner.groupby(['os_version'])['revenue'].sum().reset_index(name='revenue')
df_banner_os_sum_revenue = df_banner_os_sum_revenue.sort_values(by='revenue', ascending=False)

df_results.at['Banner','OS_Rev'] = (df_banner_os_sum_revenue.loc[df_banner_os_sum_revenue['revenue'].idxmax()])['revenue']

fig = px.bar(df_banner_os_sum_revenue, x='os_version', y='revenue')
fig.update_layout(title = "Banner revenue sum",
                  xaxis_title="os_version name",
                  yaxis_title="revenue sum")
fig.show()

Больше половины дохода от просмотра банеров дают пользователи новых моделей ОС.

Средние значения ECPM по версиям ОС

In [ ]:
# calculation of mean ECPM and count number of events for each city
df_banner_os_mean_ECPM = df_banner.groupby('os_version')['ECPM']\
                        .agg(['mean','size'])\
                        .rename(columns={'mean':'ECPM'})\
                        .reset_index()

df_banner_os_mean_ECPM = df_banner_os_mean_ECPM.loc[df_banner_os_mean_ECPM['size'] > 80]
df_banner_os_mean_ECPM = df_banner_os_mean_ECPM.sort_values(by = 'ECPM', ascending=False)

df_results.at['Banner','OS_ECPM'] = (df_banner_os_mean_ECPM.loc[df_banner_os_mean_ECPM['ECPM'].idxmax()])['ECPM']

fig = px.bar(df_banner_os_mean_ECPM, x='os_version', y='ECPM')
fig.update_layout(title = "Banner ECPM mean",
                  xaxis_title="os_version name",
                  yaxis_title="ECPM mean")
fig.show()

Для банерной рекламы кореляция модели ОС и стоимости показа так же подтверждается

## RV Ad

Из общего массива данных выделю тот что относится к rewarded video

In [ ]:
df_rv = df.loc[df['ad_type'] == 'rv']

In [ ]:
fig = go.Figure(data=[go.Histogram(y=df_rv.ECPM)])
fig.update_yaxes(range=[0, 300])
fig.update_xaxes(range=[0, 1800])
fig.update_layout(title = "ECPM RV",
                  xaxis_title="Number of cases",
                  yaxis_title="ECPM value")
fig.show()

На графике присутствуют три класса, картина схожа с той что наблюдалась в банерной рекламе. Возможно у этого есть общая причина

## Анализ по городам

Доход по городам

In [ ]:
df_rv_cities_sum_revenue = df_rv.groupby(['city'])['revenue'].sum().reset_index(name='revenue')
df_rv_cities_sum_revenue = df_rv_cities_sum_revenue.sort_values(by='revenue', ascending=False)
Total = df_rv_cities_sum_revenue['revenue'].sum()
Subtotal = df_rv_cities_sum_revenue['revenue'].iloc[:100,].sum()
df_rv_cities_sum_revenue = df_rv_cities_sum_revenue.loc[df_rv_cities_sum_revenue['revenue'] > 5]

df_results.at['RV','City_Rev'] = (df_rv_cities_sum_revenue.loc[df_rv_cities_sum_revenue['revenue'].idxmax()])['revenue']

fig = px.bar(df_rv_cities_sum_revenue, x='city', y='revenue')
fig.update_layout(title = "RV revenue sum",
                  xaxis_title="City name",
                  yaxis_title="revenue sum")
fig.show()

In [ ]:
Subtotal,Total

Первая десятка городов приносит 24.8% дохода, первая сотня 74.4%. Доход от этого типа рекламы более локализован географически чем доход от других типов. Доход от Хьюстона почти в два раза больше чем от других городов топа.

In [ ]:
# calculation of mean ECPM and count number of events for each city
df_rv_cities_mean_ECPM = df_rv.groupby('city')['ECPM']\
                        .agg(['mean','size'])\
                        .rename(columns={'mean':'ECPM'})\
                        .reset_index()

fig = px.scatter(df_rv_cities_mean_ECPM, x="ECPM", y="size")
fig.update_layout(title = "Mean ECPM to frequency RV",
                  xaxis_title="mean ECPM",
                  yaxis_title="Number of cases")
fig.show()

Размер выборки для данного типа рекламы довольно мал, что уменьшает вероятность достоверности получаемых результатов.

In [ ]:
df_rv_cities_mean_ECPM = df_rv_cities_mean_ECPM.loc[df_rv_cities_mean_ECPM['size'] > 80]

df_rv_cities_mean_ECPM = df_rv_cities_mean_ECPM.loc[df_rv_cities_mean_ECPM['ECPM'] > 3]
df_rv_cities_mean_ECPM = df_rv_cities_mean_ECPM.sort_values(by = 'ECPM', ascending=False)

df_results.at['RV','City_ECPM'] = (df_rv_cities_mean_ECPM.loc[df_rv_cities_mean_ECPM['ECPM'].idxmax()])['ECPM']

fig = px.bar(df_rv_cities_mean_ECPM, x='city', y='ECPM')
fig.update_layout(title = "RV ECPM mean",
                  xaxis_title="City name",
                  yaxis_title="ECPM mean")
fig.show()

## Анализ по версиям ОС

Суммарный доход по версиям

In [ ]:
df_rv_os_sum_revenue = df_rv.groupby(['os_version'])['revenue'].sum().reset_index(name='revenue')
df_rv_os_sum_revenue = df_rv_os_sum_revenue.sort_values(by='revenue', ascending=False)

df_results.at['RV','OS_Rev'] = (df_rv_os_sum_revenue.loc[df_rv_os_sum_revenue['revenue'].idxmax()])['revenue']

fig = px.bar(df_rv_os_sum_revenue, x='os_version', y='revenue')
fig.update_layout(title = "RV revenue sum",
                  xaxis_title="os_version name",
                  yaxis_title="revenue sum")
fig.show()

Средние значения ECPM по версиям ОС

In [ ]:
# calculation of mean ECPM and count number of events for each city
df_rv_os_mean_ECPM = df_rv.groupby('os_version')['ECPM']\
                        .agg(['mean','size'])\
                        .rename(columns={'mean':'ECPM'})\
                        .reset_index()

df_rv_os_mean_ECPM = df_rv_os_mean_ECPM.loc[df_rv_os_mean_ECPM['size'] > 80]
df_rv_os_mean_ECPM = df_rv_os_mean_ECPM.sort_values(by = 'ECPM', ascending=False)

df_results.at['RV','OS_ECPM'] = (df_rv_os_mean_ECPM.loc[df_rv_os_mean_ECPM['ECPM'].idxmax()])['ECPM']

fig = px.bar(df_rv_os_mean_ECPM, x='os_version', y='ECPM')
fig.update_layout(title = "RV ECPM mean",
                  xaxis_title="os_version name",
                  yaxis_title="ECPM mean")
fig.show()

Общий комментарий
В данном анализе я везде отображал значения наиболее успешных городов и ОС. В общем случае не менее интересны когорты с худшими параметрами. Например, они могут дать инсайты о плохом пользовательском опыте, или о рынке, в который не нужно инвестировать.
Я не стал их отображать с целью экономии времени.

In [ ]:
df_results

Можно сделать следующие выводы:
- Больше всего денег сейчас приносит полноэкранная реклама
- Самые дорогие просмотры идут от Rewarded video, самые дешевые - от банеров
- Город в большей степени чем ОС определяет доход (больше разброс значений)

Для разных типов рекламы наиболее выгодными по ECPM являются разные города,
Для Int это Cincinnati(76), Las Vegas(70), Indianapolis(67)
Для Banner это Clarks Summit (7.26), Gardena (6.75), Brockton (6.5)
Для RV это Houston (117.4) New York (69.2) Minneapolis (47.5)
Причем уже второй по счету город по стоимости Rewarded video отстает от второго города Int. 

По всем трем типам рекламы есть корреляция между версией ОС и ECPM, наиболее выгодна 12 версия ОС.

Помимо частных выводов, проведенный анализ позволяет сказать что и география, и ОС пользователя сильно влияют на доход от него, что должно учитываться при настройке таргетированной рекламы и стратегии развития продукта.

* Процент доходов содержащий информацию о городе = 62% (объяснение в Data_Preparation файле)

Идеи для дальнейшего анализа:
- На графиках ECPM есть крупные группы повышенной ценности. Есть смысл изучить каждую из них отдельно, поискать корреляции с какими либо признаками. 
- Уже предложенный в задании анализ ECPM по времени. Если есть корреляция можно настроить под нее отправку пушей напоминающих об игре.
- Есть подозрения о большой группе пользователей не приносящих доход, есть смысл их исследовать и подумать над альтернативными моделями их монетизации.